# Worksheet 05

Name: Mark Yang

UID: U27016194

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

First iteration, the 2 centroids split the input around $(0+2)/2=1$ to two partitions

`[0, .5], [1.5, 2, 6, 6.5, 7]`

Next, the points are repartitioned by their centers, $(2*0.25 + 5*4.6)/7 = 3.357$

`[0, .5, 1.5, 2], [6, 6.5, 7]`

Now, the next repartition does not change the clusters, so we know we are done.

b) Describe in plain english what the cost function for k means is.

The cost function describes how good the clustering is, where a lower cost is better. 

The cost function tries to minimize the sum of squares of the clusters, minimizing the variance of each cluster.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

The convergence algorithms are not guarenteed to find the global minimum, so different initializations produce different solutions.

d) Does Lloyd's Algorithm always converge? Why / why not?

Yes, theoretically Lloyd's algorithm always converges. 

This is because each interation minimizes the in group variance, so each iteration minimizes the global cost. Because the cost does not increase each iteration, it must eventually converge.

In practice, other effects like float precision may not guarentee the algorithm converges.

e) Follow along in class the implementation of Kmeans

In [13]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))


    def lloyds(self):
        #initialize centers
        assert len(self.data) >= self.k
        length = len(self.data)
        centers = [self.data[i] for i in np.random.choice(range(length), size=self.k, replace=False)]
        centers = np.array(centers)
        changed = True
        for iteration in range(500):
            #assign to centers
            for i, point in enumerate(self.data):
                #distance to each center
                dist = [(point[0]-c[0])**2 + (point[1]-c[1])**2 for c in centers]
                closest = np.argmin(dist)
                if closest != self.assignment[i]:
                    changed = True
                    self.assignment[i] = closest
            
            #save image
            self.snap(centers)
            
            if changed == False:
                return
            
            #compute new centers
            new_centers = [[] for c in centers]
            for i,d in enumerate(self.assignment):
                new_centers[d].append(self.data[i])
                
            for i,l in enumerate(new_centers):
                assert len(l)>0
                print("Old center:", centers[i], "New center:", np.mean(l,))
                centers[i] = np.mean(l)
            
            changed = False

kmeans = KMeans(X, 6)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)

Old center: [ 0.37642553 -1.09940079] New center: -0.8003514281671974
Old center: [1.50196755 3.92953205] New center: 2.277547568642933
Old center: [0.06651722 0.3024719 ] New center: 0.17682497836026656
Old center: [ 3.10028434 -2.70197803] New center: -1.0437574708691455
Old center: [2.38314477 0.94447949] New center: 1.810810928469467
Old center: [-4.33425847  0.65328249] New center: -0.7273157313446446
Old center: [-0.80035143 -0.80035143] New center: -0.8500044170568979
Old center: [2.27754757 2.27754757] New center: 2.573478017789184
Old center: [0.17682498 0.17682498] New center: 0.20304760165524496
Old center: [-1.04375747 -1.04375747] New center: -1.5543514469173156
Old center: [1.81081093 1.81081093] New center: 1.5418443304918223
Old center: [-0.72731573 -0.72731573] New center: -0.5284177551955175
Old center: [-0.85000442 -0.85000442] New center: -0.9082574333366583
Old center: [2.57347802 2.57347802] New center: 2.573478017789184
Old center: [0.2030476 0.2030476] New cente